# Data Prepatation



In [16]:
from __future__ import print_function
import pandas as pd
import numpy as np
import os
import pickle
import os.path
from datetime import datetime
import pyarrow
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import json

### Download and preprocess data

In [17]:
def clean_sheet_names(sheets):
    '''
    Get rid of the duplicate sheets, only take the sheets from the 
    latest point in the day
    '''
    new_ranges = []
    indices = []
    
    #Get all the tabs in the sheet 
    for s in sheets:
        new_ranges.append(s.get("properties", {}).get("title"))
        
    #split the names to just get the date
    clean_new_ranges = new_ranges.copy()
    for i, x in enumerate(clean_new_ranges):
        clean_new_ranges[i] = x.split('_')[0]    
    
    #Get the index of the latest tab for each date
    for item in set(clean_new_ranges):
        indices.append(clean_new_ranges.index(item))

    clean_new_ranges = []
    # Return wanted tabs for the sheet extraction
    for index in sorted(indices):
        clean_new_ranges.append(new_ranges[index])

    return clean_new_ranges
    

In [18]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w'

"""Shows basic usage of the Sheets API.
Prints values from a sample spreadsheet.
"""

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
        
#get all the sheet names for ranges when querying
service = build('sheets', 'v4', credentials=creds)
sheet_metadata = service.spreadsheets().get(spreadsheetId=SAMPLE_SPREADSHEET_ID).execute()
sheets = sheet_metadata.get('sheets', '')

# Clean the result to the sheet tabs we want
cleaned_ranges = clean_sheet_names(sheets)

In [19]:
cleaned_ranges

['Jan29_230pm',
 'Jan28_11pm',
 'Jan27_830pm',
 'Jan26_11pm',
 'Jan25_10pm',
 'Jan24_12pm',
 'Jan23_12pm',
 'Jan22_12pm']

In [20]:
def get_data(sheet_range):
    tmp_df = pd.DataFrame([])
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=sheet_range).execute()

    header = result.get('values', [])[0]   # Assumes first line is header!
    values = result.get('values', [])[1:]  # Everything else is data.
    
    
    # rows with no deaths and recovered vals have shorter lists
    # impute missing values with zeros
    for i, row in enumerate(values):
        if len(row) < len(header):
            extra_zeros = (len(header) - len(row))
            values[i] += [0] * extra_zeros

    # Create Dataframe
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        tmp = pd.concat(all_data, axis=1)
        
    print('...', sheet_range)
    return tmp
   
df_list = []
print('Getting sheets to preprocess')
for sheet_range in cleaned_ranges:
    df_list.append(get_data(sheet_range))

Getting sheets to preprocess
... Jan29_230pm
... Jan28_11pm
... Jan27_830pm
... Jan26_11pm
... Jan25_10pm
... Jan24_12pm
... Jan23_12pm
... Jan22_12pm


Now that we have all the data we now need to clean it 
- Fill null values
- remore suspected values
- change column names

In [21]:
def clean_data(tmp_df):
    if 'Demised' in tmp_df.columns:
        tmp_df.rename(columns={'Demised':'Deaths'}, inplace=True)

    if 'Country/Region' in tmp_df.columns:
        tmp_df.rename(columns={'Country/Region':'country'}, inplace=True)
    
    if 'Province/State' in tmp_df.columns:
        tmp_df.rename(columns={'Province/State':'province'}, inplace=True)
      
    if 'Last Update' in tmp_df.columns:
        tmp_df.rename(columns={'Last Update':'date'}, inplace=True)
        
    if 'Suspected' in tmp_df.columns:
        tmp_df = tmp_df.drop(columns='Suspected')

    for col in tmp_df.columns:
        tmp_df[col] = tmp_df[col].fillna(0)
    
    #Lower case all col names
    tmp_df.columns = map(str.lower, tmp_df.columns)    
    
    return tmp_df

cleaned_dataframes = []

print('Cleaning dataframes...')
for frame in df_list:
    cleaned_dataframes.append(clean_data(frame))

Cleaning dataframes...


In [22]:
#Impute the missing columns in the early stages with 0 values (recovered and deaths)
print('Imputing missing columns...')
cleaned_dataframes[-1]['recovered'] = [0] * (cleaned_dataframes[-1]).shape[0]
cleaned_dataframes[-1]['deaths'] = [0] * (cleaned_dataframes[-1]).shape[0]

cleaned_dataframes[-2]['recovered'] = [0] * (cleaned_dataframes[-2]).shape[0]
cleaned_dataframes[-2]['deaths'] = [0] * (cleaned_dataframes[-2]).shape[0]

print('Concatenating all sheet dataframes into one...')
final_df = pd.concat(cleaned_dataframes, sort=True)

Imputing missing columns...
Concatenating all sheet dataframes into one...


In [23]:
final_df['date'] = final_df['date'].astype(str)
final_df.head()

,confirmed,country,date,deaths,province,recovered
0,3554,Mainland China,1/29/2020 14:30,125,Hubei,88
1,296,Mainland China,1/29/2020 14:30,,Zhejiang,3
2,277,Mainland China,1/29/2020 14:30,,Guangdong,5
3,221,Mainland China,1/29/2020 14:30,0,Hunan,0
4,206,Mainland China,1/29/2020 14:30,2,Henan,1


In [24]:
final_df['date'].value_counts()

1/29/2020 14:30       54
1/27/2020 20:30       52
1/28/2020 23:00       51
1/26/2020 23:00       47
1/23/20 12:00 PM      46
1/25/2020 12:00 PM    42
1/24/2020 12:00 PM    40
1/22/2020 12:00       38
1/25/2020 10:00 PM     2
1/28/2020 18:00        1
1/24/2020 4:00 PM      1
Name: date, dtype: int64

In [41]:
# s = '1/22/2020 12:00 PM'
# dt = datetime.strptime(s,'%m/%d/%Y %H:%M PM')
# dt.date()
# dd = dt.date()
# print(dd)

def conver_date_time(date):
    
    try:
        if 'PM' in date:
            return datetime.strptime(date,'%m/%d/%Y %H:%M PM').date()
        elif 'AM' in date:
            return datetime.strptime(date,'%m/%d/%Y %H:%M AM').date()
        else:
            return datetime.strptime(date,'%m/%d/%Y %H:%M').date()
    except:
        return datetime.strptime(date, '%m/%d/%y %H:%M PM').date()


final_df['date'] = final_df['date'].apply(conver_date_time)

In [48]:
# final_df.groupby(['date']).confirmed.sum()

In [49]:
# sheets need to be sorted by date value
print('Sorting by date...')
final_df = final_df.sort_values('date')

Sorting by date...


In [50]:
#

In [51]:
print('Saving...')
file_name = './data/updated_{}.parquet.gzip'.format(datetime.date(datetime.now()))

final_df.astype(str).to_parquet(file_name, compression='gzip')

Saving...
